In [15]:
task_num="4"
var_num=1
env_step_limit=2
num_episodes=1
gpt_model="gpt_4_0613"
summarize_end_of_episode=1
device="cpu"
temperature=0.0
use_gold_memory_in_ep0=0
gold_traces=""
use_last_k_memories=3
quadrant=1
simplifications_preset="easy"
output_path_prefix = 'logs/testrun/'

In [16]:
jar_path = None
task_num = "4"
var_num = 1
env_step_limit = 100
num_episodes = 2
gpt_model = "gpt-4-0613"
summarize_end_of_episode = 1
device = None
temperature = 0.0
use_gold_memory_in_ep0 = 0
gold_traces = ""
use_last_k_memories = 3
quadrant = None
output_path_prefix = "save-histories"
simplifications_preset = None


In [4]:
task_num = int(task_num)
var_num = int(var_num)
print(f"Running CLIN agent for taskIdx:{task_num}, varIdx:{var_num}")

Running CLIN agent for taskIdx:4, varIdx:1


In [6]:
exitCommands = ["quit", "exit"]
from sentence_transformers import SentenceTransformer
sent_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens', device=device)


/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/agential/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# no gold path -_-

In [9]:
summaryfname = "logs/" + "summary.txt"
summaryFile = open(summaryfname, "w")

In [10]:
task = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring' 
print("Task Names: " + str(task))

Task Names: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring


In [11]:
"""CRITIC structured output module."""

from typing import Any, Dict, List

from pydantic import BaseModel, Field

from agential.agents.base.output import BaseAgentOutput
from agential.core.llm import Response


class ClinStepOutput(BaseModel):
    """Critic step Pydantic output class.

    Attributes:
        answer (str): The answer generated by the agent.
        critique (str): The critique of the answer generated by the agent.
        external_tool_info (Dict[str, Any]): The query requested by the agent.
        answer_response (List[Response]): The answer responses generated by the agent.
        critique_response (List[Response]): The critique responses generated by the agent.
    """

    query: str = Field(..., description="The query requested by the agent.")
    answer: str = Field(..., description="The answer generated by the agent.")
    observation: str = Field(..., description="The answer's observation.")
    external_tool_info: Dict[str, Any] = Field(
        ..., description="The external tool outputs."
    )
    answer_response: List[Response] = Field(
        ..., description="The answer responses generated by the agent."
    )
    observation_response: List[Response] = Field(
        ..., description="The observation responses generated by the agent."
    )


class ClinOutput(BaseAgentOutput):
    """Critic Pydantic output class.

    Attributes:
        additional_info (List[CriticStepOutput]): The additional info.
    """

    additional_info: List[ClinStepOutput] = Field(
        ..., description="The additional info."
    )


In [13]:
!pip install scienceworld

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 3.6 MB/s eta 0:00:00a 0:00:01m


In [17]:
jar_path, env_step_limit

(None, 100)

In [18]:
from scienceworld import ScienceWorldEnv

env = ScienceWorldEnv("", jar_path, envStepLimit=env_step_limit)

In [21]:
taskNames = env.getTaskNames()
env.load(taskNames[0], 0, "")


print("Task Description: " + str(env.getTaskDescription()))

Task Description: Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


/var/folders/s8/rtd2z3dd2nqg15b702qbw4m40000gn/T/ipykernel_94449/2966492216.py:1: UserWarning:  You are using the camel case api. This feature is deprecated. Please migrate to the snake_case api. 
  taskNames = env.getTaskNames()
/var/folders/s8/rtd2z3dd2nqg15b702qbw4m40000gn/T/ipykernel_94449/2966492216.py:5: UserWarning:  You are using the camel case api. This feature is deprecated. Please migrate to the snake_case api. 
  print("Task Description: " + str(env.getTaskDescription()))


In [ ]:
MAXTOKENSINHISTORY = 2500

In [22]:
import tokenizer
def getTokenLength(strIn):
    tokens = tokenizer.encode(strIn)
    numTokens = len(tokens)
    return numTokens


ModuleNotFoundError: No module named 'tokenizer'

In [ ]:
import openai


def call_gpt(messages,
        model_name,
        max_tokens,
        temperature):
     response = openai.ChatCompletion.create(
                model=model_name,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature
            )

In [ ]:
def generate_action(
        question , 
        previous_rationales,
        previous_actions,
        previous_observations,
        current_obs,
        model,
        summary,
        temperature,
        quadrant,
        episodeIdx
):
    next_action_query = ""

    
    sw_prompt_task += f"Below you can see the most recent history of you actions " \
                      f"to help you decide your next action."

    new_messages = [
        {"role": "system",
         "content": "You are an AI agent helping execute a science experiment "
                    "in a simulated environment with limited number of objects "
                    "and actions available at each step."
         },
        {"role": "user",
         "content": f"{sw_prompt_task}"
         }
    ]

    if len(previous_actions):

        # Hyperparameter -- the maximum tokens in the history.
        maxTokensInHistory = MAXTOKENSINHISTORY

        # Create an action history. This is the history of actions and observations that have been taken so far.
        # This is used to help the model understand what has happened so far, and what the next action should be.
        # This is built from the most-recent to least-recent, and stops when meeting a maximum token threshold.

        action_observation_history_str = ""
        previous_messages = []
        # Iterate from the last to the 0th index
        for i in range(len(previous_actions) - 1, max(len(previous_actions) - 6, -1) , -1): # we are only taking last three a-o pair to fit to the token length
            rationale = previous_rationales[i]
            action = previous_actions[i]
            observation = previous_observations[i]

            # Note we prepend at the start of list
            # So we are prepending in reverse order
            # observation after ith action, selected ith action, generated rationale for ith action
            if i < len(previous_actions) - 1:
                previous_messages[:0] = [{
                    "role": "user", "content": f"{observation}\n\nWhat action would you like to do next?"
                }]
            else:
                previous_messages[:0] = [{
                    "role": "user", "content": f"{observation}"
                }]

            previous_messages[:0] = [{
                "role": "user", "content": f"Selected action: {action}"
            }]

            previous_messages[:0] = [{
                "role": "assistant", "content": f"{rationale}"
            }]

            action_observation_history_str_candidate = f"assistant: {rationale} step: {i}\n action: {action}\n observation: {observation}\n\n" + action_observation_history_str

            # If we have reached the maximum number of tokens, stop.
            if getTokenLength(action_observation_history_str_candidate) > maxTokensInHistory:
                break
            else:
                action_observation_history_str = action_observation_history_str_candidate

        for prev_message in previous_messages:
            new_messages.append(prev_message)

    next_actions_set = [x for x in next_actions_set if 'focus' not in x]
    actions_str = '\n '.join(next_actions_set)

    next_action_query += f"Here is what is the current observation:" \
                 f"\n" \
                 f" {current_obs}" \
                 
    next_action_query += f"\nYour next action should be in one of the following formats:" \
                 f"\nPossible actions:" \
                 f"\n {actions_str}" \
                 f"\n\n" \
                 f"If I say \"Ambiguous request\", your action might mean multiple things. In that case, respond with the number corresponding to the action you want to take.\n" \
                 f"\n\n" \
                 f"What action would you like to do next?\n" \
                 f"First, scan the (unordered) list of learnings, if provided. Decide if any of the learnings are applicable given the last observation to make progress in this task. Then only use selected learnings, if any, to construct a rationale for picking the next action. If no Learning is selected, construct the rationale based on the last observation. Format your response as follows:\n" \
                 f"Write 'I used learning id(s):' as a comma separated list; the list can be empty if no learnings selected. Then, write $$$ followed by the rationale. Finally, write ### followed by the single next action you would like to take." \
                 f"If you think you have completed the task, please write TASK_COMPLETE as the next action." \
                 f"If the task requires you to 'focus' on something (OBJ), please write FOCUS ON <OBJ> as the next action. FOCUS is a extremely critical action that can be only used the number of times 'focus' is mentioned in the task description. Using it more than that or inappropiately (such as on a wrong object) will terminate the session and the task will be rendered as incomplete." \
                 f"If you performed an action that requires waiting to see the effect, please write 'wait' as the next action."  \
    
    

    new_messages.append({
        "role": "user", "content": f"{next_action_query}"
    })

    prompt_str = [rec['role'] + ": " + rec['content'] for rec in new_messages]
    # print(
        # f"ChatGPT prompt:\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n{prompt_str}\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    response = call_gpt(
        messages=new_messages,
        model_name=model,
        max_tokens=256,
        temperature=temperature,  # 0 for greedy best decoding       # PJ modified from 0.7 to 0.0
    )
    # print(f"current_state:{current_obs}")
    # print(f"=====\nNEXT ACTION \ntask:{task}\n current_state:{current_state}"
    #       f"\nobjects_set: {objects_set}\nnext_actions_set:{next_actions_set}")
    # print(f"response:{response}")
    # if out_logs_file:
    #     out_logs_file.write(f"next action\t{prompt_str}\t{json.dumps(response)}")

    # Sometimes ChatGPT returns a long string with actions mentioned in ""
    # Extract strings within double quotes

    response_str = response['choices'][0]['message']['content']
    response['response_str'] = response_str
    # print("RAW RESPONSE STRING:")
    # print(response_str)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

    # next_actions = re.findall('"([^"]*)"', response_str)
    ## If no such quoted actions found, consider entire generation as next action
    # if not next_actions:
    #    next_actions = [response['choices'][0]['message']['content']]

    # actions should be between "###" blocks. Take the first one (in case it generated multiple ones) as the next action
    possibleActions = response_str.split("###")
    reasoningStr = " "
    next_action_str = " "
    if len(possibleActions) > 1:
        reasoningStr = possibleActions[0]
        next_action_str = possibleActions[1].lower().strip()  # The first index should be it's reasoning, the second should be it's action.
        # It's possible it might put multiple actions in the next_action_str, that are comma delimited. Trim out all but the first one.
        # next_action_str = next_action_str.split(",")[0].lower().strip()

    # next_action_str = next_actions[0].lower().strip()
    next_action_str = next_action_str.replace(".", "").replace("i would like to ", "") # .split(' and ')[0]
    response['pred_next_action'] = next_action_str

    # Check to make sure the reasoningStr and next actions are not blank (to prevent the data structure from crashing with blank strings)
    if len(reasoningStr) < 1:
        reasoningStr = " "
    if (len(next_action_str) < 1):
        next_action_str = " UNKNOWN "

    response['reasoningStr'] = reasoningStr

    # Append ChatGPT response and our action selection to message history
    new_messages.append({
        "role": "assistant", "content": response_str})
    new_messages.append({
        "role": "user", "content": f"Selected action: {next_action_str}"
    })
    # print(f"pred_next_action:{next_actions[0]}")

    return response

In [ ]:
from typing import Tuple
from agential.utils.parse import remove_newline


def generate_observation(
        self, idx: int, scratchpad: str, action_type: str, query: str
    ) -> Tuple[str, str, str, bool, Dict[str, Any]]:
        """Generates an observation based on the provided action type and query.

        Args:
            idx (int): The index of the current observation.
            scratchpad (str): The current state of the scratchpad.
            action_type (str): The type of action performed (e.g. "search", "lookup", "finish").
            query (str): The query for the action.

        Returns:
            Tuple[str, str, str, bool, Dict[str, Any]]: The updated scratchpad, the answer, the observation, a flag indicating if the task is finished, and a dictionary containing external tool information.
        """
        answer = ""
        finished = False
        external_tool_info = {"search_result": "", "lookup_result": ""}

        scratchpad += f"\nObservation {idx}: "
        if action_type.lower() == "finish":
            answer = query
            finished = True
            obs = query
        elif action_type.lower() == "search":
            try:
                search_result = self.docstore.search(query)
                external_tool_info["search_result"] = search_result
                obs = remove_newline(search_result)
            except Exception:
                obs = "Could not find that page, please try again."
        elif action_type.lower() == "lookup":
            try:
                lookup_result = self.docstore.lookup(query)
                external_tool_info["lookup_result"] = lookup_result
                obs = remove_newline(lookup_result)

            except ValueError:
                obs = "The last page Searched was not found, so you cannot Lookup a keyword in it. Please try one of the similar pages given."
        else:
            obs = "Invalid Action. Valid Actions are Lookup[<topic>] Search[<topic>] and Finish[<answer>]."
        scratchpad += obs

        return scratchpad, answer, obs, finished, external_tool_info


In [ ]:
import tiktoken
from agential.utils.docstore import DocstoreExplorer
from tiktoken.core import Encoding
from agential.core.llm import BaseLLM
from langchain_community.docstore.wikipedia import Wikipedia

class ClinQAStrategy():
    def __init__(
        self,
        llm: BaseLLM,
        max_iteration: int = 6,
        max_tokens: int = 5000,
        enc: Encoding = tiktoken.encoding_for_model("gpt-3.5-turbo"),
        docstore: DocstoreExplorer = DocstoreExplorer(Wikipedia()),
        testing: bool = False,
    ) -> None:
        """Initialization."""
        self.max_iteration = max_iteration
        self.max_tokens = max_tokens
        self.enc = enc
        self.docstore = docstore
        self.testing = testing
    
    def generate(
        self,
        question: str,
        model: str = "gpt-3.5-turbo",

    ): 
   
        summary = ""

        prev_episode_summary_str = ""

        previous_actions = []
        previous_observations = []
        previous_rationales = []
        # Start running episodes
        for episodeIdx in range(0, self.max_iteration):
            # Run histories
            runHistories = []
            score = 0.0
            score_positive = 0.0
            isCompleted = False
            curIter = 0
            observation = ""
            for i in range(0, self.max_iteration):
                
                # Generate actio
                action = generate_action(
                    question , 
                    previous_rationales,
                    previous_actions,
                    previous_observations,
                    observation,
                    model,
                    summary,
                    temperature,
                    quadrant,
                    episodeIdx
                )
                
                generated_rationale_str = action['response_str']
                generated_action_str = action["pred_next_action"].lower().strip()
                print("GPT4 generated action: " + str(generated_action_str))
                observation , reward , isCompleted , info = generate_observation(question , action , summary)

                if isCompleted:
                    break
                       
                # Reasoning rationale
                print("Reasoning rationale: " + str(generated_rationale_str))
                previous_rationales.append(action["reasoningStr"])

                print("Action: " + str(action["pred_next_action"]))
                previous_actions.append(generated_action_str)

                print("Observation: " + str(observation))
                previous_observations.append(observation)



In [ ]:
from agential.agents.base.agent import BaseAgent
from agential.core.llm import BaseLLM

class Clin(BaseAgent):
    """Clin Agent.

    Attributes:
        llm (BaseLLM): An instance of a language model used for generating initial answers
            and critiques.
        benchmark (str): The benchmark.
        testing (bool): Whether to run in testing mode. Defaults to False.
        **strategy_kwargs (Any): Additional strategy-specific arguments.
    """

    def __init__(
        self,
        llm: BaseLLM,
        benchmark: str,
        testing: bool = False,
        **strategy_kwargs: Any,
    ) -> None:
        """Initialization."""
        super().__init__(llm=llm, benchmark=benchmark, testing=testing)

        self.strategy = Clin.get_strategy(
            benchmark=self.benchmark, llm=self.llm, testing=testing, **strategy_kwargs
        )


In [ ]:
num_test_variations_to_run = 1
for 